## Trying pmc_id_converter

> "pmc_id_converter for ID converter between PMID, PMCID and DOI"

Here the demonstration of this package is done in temporary sessions served via the MyBinder service, and so if you make anything useful save it back to your local machine ASAP.

--------

# Use `pmc_id_converter` via Command Line interface

NOTE: 
In these demonstrations the exclamation point is used to send commands to the command line. You would not not need this in the terminal/console. **In other words, in most cases you'll want to actually delete the exclamation point shown at the start of these commands to use on the REAL command line.**

Running `pmc_idconv --help` shows you several command examples and options for usage

In [1]:
!pmc_idconv --help

Usage: pmc_idconv [OPTIONS] [IDS]...

  ID converter between PMID, PMCID and DOI

Options:
  --version           Show the version and exit.
  -o, --outfile TEXT  the output filename [stdout]
  -h, -?, --help      Show this message and exit.

  

  Examples:
      pmc_idconv --help
      pmc_idconv 30003000                     [PMID]
      pmc_idconv PMC6039336                   [PMCID]
      pmc_idconv 10.1007/s13205-018-1330-z    [DOI]
      pmc_idconv 30003000 30003001 30003002   [BATCH]
      pmc_idconv 30003000 30003001 -o out.jl  [FILE]  

  Contact: suqingdong <suqingdong1114@gmail.com> 


Let's step through running those examples handling identifier conversion here as a way to cover the use of this on the command line.

In [4]:
!pmc_idconv 30003000

{"doi": "10.1007/s13205-018-1330-z", "pmcid": "PMC6039336", "pmid": 30003000, "requested-id": "30003000"}


In [5]:
!pmc_idconv PMC6039336

{"doi": "10.1007/s13205-018-1330-z", "pmcid": "PMC6039336", "pmid": 30003000, "requested-id": "PMC6039336"}


In [6]:
!pmc_idconv 10.1007/s13205-018-1330-z 

{"doi": "10.1007/s13205-018-1330-z", "pmcid": "PMC6039336", "pmid": 30003000, "requested-id": "10.1007/s13205-018-1330-z"}


The fourth example involving identifiers demonstrates BATCH conversion of multiple identifiers.

In [7]:
!pmc_idconv 30003000 30003001 30003002

{"doi": "10.1007/s13205-018-1330-z", "pmcid": "PMC6039336", "pmid": 30003000, "requested-id": "30003000"}
{"doi": "10.1002/open.201800095", "pmcid": "PMC6031859", "pmid": 30003001, "requested-id": "30003001"}
{"doi": "10.1002/open.201800044", "pmcid": "PMC6031856", "pmid": 30003002, "requested-id": "30003002"}


The fifth example from the usage involving identifiers is meant to demonstrate sending the the results to a file. The related example on the repo [there]((https://github.com/suqingdong/pmc_id_converter#command-line) looks more fully realized version of this and so I suggest using that instead:

```shell
# Output to a file
pmc_idconv 30003000 30003001 30003002 -o out.json
```

In [9]:
!pmc_idconv 30003000 30003001 30003002 -o out.json

[2025-10-22 15:03:42 Main cli DEBUG MainThread:45] save file to: out.json


At this time I cannot explain why it shows `DEBUG MainThread:45` because everything seems to work so I suggest ignoring that for now.  
Run the next command to demonstratie the last example worked:

In [12]:
!head out.json

{"doi": "10.1007/s13205-018-1330-z", "pmcid": "PMC6039336", "pmid": 30003000, "requested-id": "30003000"}
{"doi": "10.1002/open.201800095", "pmcid": "PMC6031859", "pmid": 30003001, "requested-id": "30003001"}
{"doi": "10.1002/open.201800044", "pmcid": "PMC6031856", "pmid": 30003002, "requested-id": "30003002"}


----------------

### Get some data to use:

I happen to have some data in fastq format available as part of [my SICILIAN-binder repo](https://github.com/fomightez/SICILIAN-binder), so I'll use that for now.   
FIX THIS SOON!!! ===> **THIS ISN'T PAIRED DATA SO DOESN'T REALLY ADDRESS `fqgrep  --paired` AND SINCE THAT IS AN IMPORTANT FEATURE, IT ISN'T BEST FOR DEMO!!!**

In [2]:
!curl -OL https://github.com/fomightez/SICILIAN-binder/raw/refs/heads/master/demo_input/chr21_mapped_SRR12269911.fastq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 4467k  100 4467k    0     0  2763k      0  0:00:01  0:00:01 --:--:-- 2763k


Let's rename it something more streamlined.

In [3]:
!mv chr21_mapped_SRR12269911.fastq demo.fastq

Now let's try it...

-----

### Usage

Show 'Usage':

In [4]:
!fqgrep --help

OVERVIEW

The fqgrep utility searches any given input FASTQ files, selecting records whose bases match one or more patterns.  By default, a pattern matches the bases in a FASTQ record if the regular expression (RE) in the pattern matches the bases.  An empty expression matches every line.  Each FASTQ record that matches at least one of the patterns is written to the standard output.

INPUT COMPRESSION

By default, the input files are assumed to be uncompressed with the following exceptions: (1) If the input files are real files and end with .gz or .bgz, they are assumed to be GZIP compressed, or (2) if they end with .fastq or .fq, they are assumed to be uncompressed, or (3) if the -Z/--decompress option is specified then any unrecongized inputs (including standard input) are assumed to be GZIP compressed.

THREADS

The --threads option controls the number of threads used to search the reads. Independently, for single end reads or interleaved paired end reads, a single thread will be us

Of course, if really running on command line and not in IPython or a Jupyter Notebook, leave off the exclamation point:

```shell
fqgrep --help
```

Basic fqgrep with a sequence, to match a single pattern in a single FASTQ file:

In [5]:
!fqgrep 'ACAAGACTGG' ./demo.fastq

@SRR12269911.276027
ATTATGAAGTGTTCCCAGTGCCACACCGTTGAAAAGGGAGGCAAGCACAAGACTGGGCCAAATCTCCATGGGCTCT
+
AAAAAEEEEEE/E/EEEEEEEEEEE/AEAAE/EEEE6EEEEEEEEE/E/EE/EA//EAEEEA<EEEE//E</EE/<
@SRR12269911.389384
ATGAAGTGTTCCCAGTGCCACACCGTTGAAAAGGGAGGAAAGCACAAGACTGGGCCAAATATCCATGGTCTCTTTT
+
A/AAAEEEEEEEAE/AEAEEEEEEA/EE//AE6EAEEE//A/AE/EEEEEE//E6E//EA/EE/EE/E<EEAEEE/
@SRR12269911.415017
ATGAAGTGTTCCCAGTGCCACACCGTTGAAAAGGGAGGCAAGCACAAGACTGGGCCAAATCTCCATGGTCTCTTTG
+
AAA/AEEEEEEAEEEAE/EAE/E<EAAEEA6EEEEAEEEE/AE/EEEEE<AEAEAAEE/AEE6EEEEEA6AEEEE/
@SRR12269911.281462
GTGCCACACCGTTGAAAAGGGAGGCAAGCACAAGACTGGGCCAAATCTCCATGGTCTCTTTGGGCGGAAGACAGGT
+
AAAAAEEEEEEEEEAEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEAEAEEEEEEEEEEEEEEEEEEEEEEEEE
@SRR12269911.281954
GTGCCACACCGTTGAAAAGGGAGGCAAGCACAAGACTGGGCCAAATCTCCATGGTCTCTTTGGGCGGAAGACAGGT
+
AAAAAEE/EEEEEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEAEEEAEEEEEE<EEEEEEEAEEEEEE/EEAEEE


fqgrep offes several options. One is that you can use the `--reverse-complement` flag to **also** search the reverse complement for matches.

In [6]:
!fqgrep --reverse-complement 'ACAAGACTGG' ./demo.fastq

@SRR12269911.287580
GCCCAGTCTTGTGCTTGACTCCCTTTTCAACGGTGTGGCACTGGGAACACTTCATAATAAAAATCTTCTTGAATTT
+
6AA/AEE/AAEEEA</A//</EEEE/E/EA//E/<E//AEEA/AEAE<//</E/E//AAA/EE/<AEE/6///EEE
@SRR12269911.99617
CAAAGAGACCATGGAGATTTGGCCCAGTCTTGTGCTTGCCTCCCCTTTCAACGGTGTGGCACTGGGAACACTTCAT
+
AAAAAEEEEAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEAEAEEEEEE6EEEEEE
@SRR12269911.276027
ATTATGAAGTGTTCCCAGTGCCACACCGTTGAAAAGGGAGGCAAGCACAAGACTGGGCCAAATCTCCATGGGCTCT
+
AAAAAEEEEEE/E/EEEEEEEEEEE/AEAAE/EEEE6EEEEEEEEE/E/EE/EA//EAEEEA<EEEE//E</EE/<
@SRR12269911.389384
ATGAAGTGTTCCCAGTGCCACACCGTTGAAAAGGGAGGAAAGCACAAGACTGGGCCAAATATCCATGGTCTCTTTT
+
A/AAAEEEEEEEAE/AEAEEEEEEA/EE//AE6EAEEE//A/AE/EEEEEE//E6E//EA/EE/EE/E<EEAEEE/
@SRR12269911.415017
ATGAAGTGTTCCCAGTGCCACACCGTTGAAAAGGGAGGCAAGCACAAGACTGGGCCAAATCTCCATGGTCTCTTTG
+
AAA/AEEEEEEAEEEAE/EAE/E<EAAEEA6EEEEAEEEE/AE/EEEEE<AEAEAAEE/AEE6EEEEEA6AEEEE/
@SRR12269911.281462
GTGCCACACCGTTGAAAAGGGAGGCAAGCACAAGACTGGGCCAAATCTCCATGGTCTCTTTGGGCGGAAGACAGGT
+
AAAAAEEEEEEEEEAEEEEEEE

Note that in addition to five results seen without the `--reverse-complement` flag, an additional three, the first two and last one -- `@SRR12269911.287580` , `@SRR12269911.99617`, and `@SRR12269911.189365`--, are **seen with the flag**.

Note that [patmatch](https://github.com/fomightez/patmatch-binder#patmatch-binder) can also give you the reverse complement match. There you use `-c` flag. Unlike fqgrep though, patmatch can also handle matches with mismatches by setting numbers of insertions, deletions, and substitutions allowed.

For being able to tell where the match occurs, the `--color` flag is really nice.

In [7]:
!fqgrep --reverse-complement --color auto 'ACAAGACTGG' ./demo.fastq

@SRR12269911.287580
GCCCAGTCTTGTGCTTGACTCCCTTTTCAACGGTGTGGCACTGGGAACACTTCATAATAAAAATCTTCTTGAATTT
+
6AA/AEE/AAEEEA</A//</EEEE/E/EA//E/<E//AEEA/AEAE<//</E/E//AAA/EE/<AEE/6///EEE
@SRR12269911.99617
CAAAGAGACCATGGAGATTTGGCCCAGTCTTGTGCTTGCCTCCCCTTTCAACGGTGTGGCACTGGGAACACTTCAT
+
AAAAAEEEEAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEAEAEEEEEE6EEEEEE
@SRR12269911.276027
ATTATGAAGTGTTCCCAGTGCCACACCGTTGAAAAGGGAGGCAAGCACAAGACTGGGCCAAATCTCCATGGGCTCT
+
AAAAAEEEEEE/E/EEEEEEEEEEE/AEAAE/EEEE6EEEEEEEEE/E/EE/EA//EAEEEA<EEEE//E</EE/<
@SRR12269911.389384
ATGAAGTGTTCCCAGTGCCACACCGTTGAAAAGGGAGGAAAGCACAAGACTGGGCCAAATATCCATGGTCTCTTTT
+
A/AAAEEEEEEEAE/AEAEEEEEEA/EE//AE6EAEEE//A/AE/EEEEEE//E6E//EA/EE/EE/E<EEAEEE/
@SRR12269911.415017
ATGAAGTGTTCCCAGTGCCACACCGTTGAAAAGGGAGGCAAGCACAAGACTGGGCCAAATCTCCATGGTCTCTTTG
+
AAA/AEEEEEEAEEEAE/EAE/E<EAAEEA6EEEEAEEEE/AE/EEEEE<AEAEAAEE/AEE6EEEEEA6AEEEE/
@SRR12269911.281462
GTGCCACACCGTTGAAAAGGGAGGCAAGCACAAGACTGGGCCAAATCTCCATGGTCTCTTTGGGCGGAAGACAGGT
+
AAAAAEEEEEEEEEAEEEEEEE

The difference in the settings seems to be that `auto` turns off the export of the color if you send it to file because you probably don't want the special codes in there that make it possible to specify the color. That is because you are probably using the output for downstream use. If you are just using it for visualization though, you can opt to keep the color syntax code. This will be demonstrated below in the course of demonstrating combining with Python and convenient Jupyter abilities.

----

### Combine with Python / Jupyter Conveniences

Of course, when running in Jupyter with an ipykernel, fqgrep use can be combined in code to take advantage of Python variables and additional available Jupyter conveniences, like so for instance:

In [8]:
my_seq = "GAGGCAAC"
!fqgrep --color auto '{my_seq}' ./demo.fastq

@SRR12269911.51990
CCTCAAGTGGGTCCCTGACCCACAAGTAGCTTAACTGGGAGGCAACCCCCAGTAGGGGCAGACTGACACCTCACAC
+
AAAAAEEEEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEE6EEE//EEEEEEEAEEEEEE/EAEAE/E6EEE</E/E
@SRR12269911.195627
CCTGATAAGATCTCAGGAGTTGGGTGAGTGGGCTCAAGCATGTGCACTAAGAGGCAACATGGCAGAGTTTAAGTGG
+
AA6AAEA6EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE/EA/EEEEEE/EEEEEE/EEAEEE/EAEEA
@SRR12269911.374754
GTTAACCTCACTCCACGTGTGTCTGTGTCCTTAATTTCCTTGGCATGAGGCAACGAATCTCGGGTATTACCCCAGA
+
AAAAAEEEEEEEEEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEEEEEEAEAEAEAEEEEAEEEEEAEEEE
@SRR12269911.158183
CTGTAATCCCAGCACTTTGGAAGGCTGAGGTGAGAGGATCACTTGAGCCCAGTAGTTTGAGGCAACATAGTGAGAT
+
AAAAAEEEEEEEEEEAEEEEEEEEEAEEEEEEEEEEEEEEEEEEEEEEAAEEEEEAAEEEEEEEEEEEEEEEEEEE
@SRR12269911.258882
ATGTGACTGGAGTGAGCAGGGGAGAGGCAACCAACTCAGGAGTCAGATCCCTGGCCAGACTAAAAGCGAACATTCA
+
AAAAAEEEEEEEEEEEEEEEEEEEEEEEE/EEEAEEEEEEEEEEEE/EEEEEEEEAEEEEE/EE/EEEEEAEEEE<
@SRR12269911.24121
GCTCAGAGGCAACTTCCTCTTCCCTTGAACTCCGGATCAGCCTTTCCAGGTGCTTTAGTCGTTCGCCTAGACACGT
+
AAAAAEEEEEEAEEEAEEEAEEE

The ability that offers starts to touch on why you'd want to combine like that.
For example, imagine if you had several sequences to search.    
Well you can use something like:

```
!fqgrep --regexp 'GCGATTAA' --regexp 'GACATTAA' ./demo.fastq
```

But imagine for downstream use, it was important you knew which was derived from each sequence.  
So you could do this:

In [9]:
my_seq1 = "GCGATTAA"
my_seq2 = "GACATTAA"
seq_list = [my_seq1,my_seq2]
for the_seq in seq_list:
    !fqgrep --color always '{the_seq}' ./demo.fastq > {the_seq}_search_results.txt

Now to look at the results:

In [10]:
!cat GCGATTAA_search_results.txt

@SRR12269911.77594
ATATCAGGTTGCAGCGGCAGAGAGCCATCCTCTATAAGCGGCTGAAAGTGCCTCCTGCGATTAACCAGTTCACCCA
+
AAAAAEEEEEEEEEEEEEEEEEEEEEAEEEEEEEEEAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEE/
@SRR12269911.248887
ATATCAGGTTGCAGCGGCAGAGAGCCATCCTCTATAAGCGGCTGAAAGTGCCTCCTGCGATTAACCAGTTCACCCA
+
AAAAAEEEEEEEEEEEEEAEEEEEEE/EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEEAEEE<


In [11]:
!cat GACATTAA_search_results.txt

@SRR12269911.223267
CCCGTACACTGTGCTTGGACAAATACCTGCTAGTGCTGCTTGGTTCACATGCTTGGACATTAAAGATGCATTCTTC
+
AAAAAEEEEEEEEEAEEE/EEEEE/EEEEEEEEEEEEEEE6EEEEEAEEEEEEEEEEAAEE6E/EEAEEEEEA<E<
@SRR12269911.98049
CAGAAACTGGTTTGGACTAAGAAAGCCAGAGTGCAGAATCTGGATGAGGTGAAACCGACATTAATAAACCTCCAAG
+
AAAAAEEEEEEEEEEEEEEEEAEEEEEAEEEEEEEEEEEEEEEEEEEEEEEEEAEEE/EEEEEEEEEEEEEEEEEE
@SRR12269911.396878
CCTGACATTAAATAAAACTCCAAAAATTAAATTCTGGCCCTCAAACCCCACAACAGGACTTAATTAACCCTGTCTC
+
AAAAAE6EE6EEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEAE/EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEA
@SRR12269911.373844
AGACTTTTCTTTGAAGGATGACTACAGACATTAAATAATCGAAGTAATTTTGGGTGGGGAGAAGAGGCAGATTCAA
+
AAAAAEEEE/E/EEE/EEEE/EEEEAEEAE/EEEAEEEEEEAEEEAEE/E/AEEEEEAEEAE/AAEEEEAEEE</E
@SRR12269911.293306
AAATAACCCCGGGCAAGACTTTTCTTTGAAGGATGACTACAGACATTAAATAATCGAAGTAATTTTGGGTGGGGAG
+
AAAAAEEEEEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEEEEEAEEEEEE
@SRR12269911.176846
GTATTACATAAATAAATTAAATAAAATAACCCCGGGCAAGACTTTTCTTTGAAGGATGACTACAGACATTAAATAA
+
AAAAAEAEEAEEE6EEEEEEEE

`GCGATTAA_search_results.txt` only has two matches, and it would have been hard to find if we mixed them in.

Of course, we could have just ran the searches in turn; however, imagine instead of two sequences, we had ten to search, and then quickly it becomes apparent how much convenience this offers.

Note that in the course of demonstrating integrating Python and Jupyter conveniences, we used  `--color always` instead of `--color auto` because even though we were sending the output to a file, we were only planning to visualize it and so we wanted to color coding included. In most cases when sending to file, you'd want `--color auto` as touched upon when introducing the `--color` flag. Otherwise the color codes will be in there as you can see if you right-click on the file `GCGATTAA_search_results.txt` in the file browser pane on the left and open it.

### Regular Expressions as the Pattern

Can use regular expressions.  
The next two cells will show that. The code only differs by the additon of a `?` and the regular expressions options flag:

In [12]:
!fqgrep --color never --regexp 'GCGA?TTAA' ./demo.fastq

@SRR12269911.288516
AGTCTAGAGTGTTTGCTTTATTCATGTTGTTTATGCGTTAAAATATAAGCATATTGCATACATAAGCATATTTACA
+
AAAAAAEEEEEEAEEEEEEEEEEEEEEEEEEEEEEAEEEAEEEEEEEEE<EEEEEEEEEEEEEEEE<EEEEEEEE/
@SRR12269911.398076
AGTCTAGAGTGTTTGCTTTATTCATGTTGTTTATGCGTTAAAATATAAGCATATTGCATACATAAGCATATTTACA
+
A/A/AAEEEEEAAAEEEEEEEE/E///AA/AEEEEEE/EEAE/EEEE/E<EEEEE/<E/AEEEEEA/EE///<A//
@SRR12269911.77594
ATATCAGGTTGCAGCGGCAGAGAGCCATCCTCTATAAGCGGCTGAAAGTGCCTCCTGCGATTAACCAGTTCACCCA
+
AAAAAEEEEEEEEEEEEEEEEEEEEEAEEEEEEEEEAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEE/
@SRR12269911.248887
ATATCAGGTTGCAGCGGCAGAGAGCCATCCTCTATAAGCGGCTGAAAGTGCCTCCTGCGATTAACCAGTTCACCCA
+
AAAAAEEEEEEEEEEEEEAEEEEEEE/EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEEAEEE<


In [13]:
!fqgrep --color auto 'GCGATTAA' ./demo.fastq

@SRR12269911.77594
ATATCAGGTTGCAGCGGCAGAGAGCCATCCTCTATAAGCGGCTGAAAGTGCCTCCTGCGATTAACCAGTTCACCCA
+
AAAAAEEEEEEEEEEEEEEEEEEEEEAEEEEEEEEEAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEE/
@SRR12269911.248887
ATATCAGGTTGCAGCGGCAGAGAGCCATCCTCTATAAGCGGCTGAAAGTGCCTCCTGCGATTAACCAGTTCACCCA
+
AAAAAEEEEEEEEEEEEEAEEEEEEE/EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEEAEEE<


See the difference?  
`--regexp 'GCGA?TTAA'` allows matches to `GCGTTAA` or `GCGATTAA`.

------

Enjoy!